In [1]:
import sys
sys.path.append("/Users/ruinz/.pyenv/versions/3.9.5/lib/python3.9/site-packages")
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
from IPython.display import clear_output

In [2]:
def make_df(artist_name, artist_number, page_count):  
    list_df = pd.DataFrame(columns=['曲名','発売日', '表示回数', '歌詞']) 
    for page in range(1,page_count+1): # アーティストの歌詞一覧のページ数に応じて設定。米津玄師の場合は1ページのみ
        base_url = 'https://www.uta-net.com'

        #歌詞一覧ページ
        url = f'https://www.uta-net.com/artist/{artist_number}/0/' + str(page) + '/'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        links = soup.find_all('td', class_='side td1')

        for link in links:
                a = base_url + (link.a.get('href'))

                #歌詞詳細ページ
                response = requests.get(a)
                soup = BeautifulSoup(response.text, 'lxml')

                # 曲名を取得
                song_name = soup.find('h2').text

                # 発売日、表示回数などを取得
                detail = soup.find('p', class_="detail").text

                # 発売日を取得
                match = re.search(r'\d{4}/\d{2}/\d{2}', detail)
                release_date = datetime.strptime(match.group(), '%Y/%m/%d').date()

                # 表示回数を取得
                p = r'この曲の表示回数：(.*)回'
                impressions = re.search(p, detail).group(1)

                # 歌詞を取得
                song_lyrics = soup.find('div', itemprop='lyrics')
                print(type(song_lyrics))
                # song_lyrics = song_lyrics.replace("<br/>", "。")
                song_lyric = song_lyrics.text
                song_lyric = song_lyric.replace('\n','')
                song_lyric = song_lyric.replace('\r', "。")
                song_lyric = song_lyric.replace('この歌詞をマイ歌ネットに登録 >このアーティストをマイ歌ネットに登録 >','')

                #サーバーに負荷を与えないため1秒待機
                time.sleep(1)

                #取得した歌詞を表に追加
                tmp_se = pd.DataFrame([[song_name], [release_date], [impressions], [song_lyric]], index=list_df.columns).T
                list_df = list_df.append(tmp_se)
                list_df.to_csv(f'./csv/{artist_name}.csv')
# clear_output()

In [5]:
make_df("mom", 26679, 1)
clear_output()

In [4]:
!pwd
!ls 

/Users/ruinz/Desktop/スクレイピング/scraping/lyric_analysis
csv                    lyric_word_cloud.ipynb
fonts                  make_dataframe.ipynb
